In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd


c:\Users\vilch\OneDrive\Escritorio\5TH SEMESTER CODE\MACHINE LEARNING\PROYECTO_4\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Comprobar si CUDA está disponible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Cargar los datos
df = pd.read_csv('train.csv')

# Verificar las columnas del DataFrame
print(df.head())
print(df.columns)

# Convertir las etiquetas a valores numéricos
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

# Dividir el dataset en conjunto de entrenamiento y prueba
train_texts, val_texts, train_labels, val_labels = train_test_split(df['message'], df['label'], test_size=0.2, random_state=42)

# Restablecer los índices de los conjuntos de datos
train_texts = train_texts.reset_index(drop=True)
val_texts = val_texts.reset_index(drop=True)
train_labels = train_labels.reset_index(drop=True)
val_labels = val_labels.reset_index(drop=True)

# Verificar los tamaños de los conjuntos de datos
print(f"Train texts: {len(train_texts)}, Validation texts: {len(val_texts)}")
print(f"Train labels: {len(train_labels)}, Validation labels: {len(val_labels)}")

# Cargar el modelo y el tokenizador preentrenado
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Tokenizar los textos
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=128)

# Crear un dataset personalizado en PyTorch
class SentimentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)  # Asegurarse de que las etiquetas sean LongTensor
        return item

train_dataset = SentimentDataset(train_encodings, train_labels)
val_dataset = SentimentDataset(val_encodings, val_labels)

# Definir los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir='./results',          # Carpeta de salida para los resultados
    num_train_epochs=3,              # Número de épocas de entrenamiento
    per_device_train_batch_size=16,  # Tamaño del batch para entrenamiento
    per_device_eval_batch_size=64,   # Tamaño del batch para evaluación
    warmup_steps=500,                # Pasos de calentamiento para el scheduler
    weight_decay=0.01,               # Peso de decaimiento
    logging_dir='./logs',            # Carpeta de salida para los logsS
    evaluation_strategy="epoch",     # Estrategia de evaluación en cada época
)

# Crear el Trainer
trainer = Trainer(
    model=model,                         # El modelo a entrenar
    args=training_args,                  # Argumentos de entrenamiento
    train_dataset=train_dataset,         # Dataset de entrenamiento
    eval_dataset=val_dataset,            # Dataset de validación
    compute_metrics=lambda p: {'accuracy': accuracy_score(p.label_ids, p.predictions.argmax(-1))},  # Métrica de precisión
)

# Entrenar el modelo
trainer.train()

# Evaluar el modelo
eval_result = trainer.evaluate()

# Generar predicciones y el reporte de clasificación
predictions = trainer.predict(val_dataset)
pred_labels = predictions.predictions.argmax(-1)

accuracy = accuracy_score(val_labels, pred_labels)
class_report = classification_report(val_labels, pred_labels, target_names=label_encoder.classes_)

print(f'Accuracy: {accuracy:.4f}')
print('Classification Report:')
print(class_report)

Using device: cuda
                                             message label
0  I saw this movie in NEW York city. I was waiti...   neg
1  This is a German film from 1974 that is someth...   neg
2  I attempted watching this movie twice and even...   neg
3  On his birthday a small boys tells his mother ...   neg
4  The person who wrote the review "enough with t...   pos
Index(['message', 'label'], dtype='object')
Train texts: 20000, Validation texts: 5000
Train labels: 20000, Validation labels: 5000


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\vilch\OneDrive\Escritorio\5TH SEMESTER CODE\MACHINE LEARNING\PROYECTO_4\venv\lib\site-packages\transformers\training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  0%|          | 0/3750 [00:00<?, ?it/s]c:\Users\vilch\OneDrive\Escritorio\5TH SEMESTER CODE\MACHINE LEARNING\PROYECTO_4\venv\lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = to

{'loss': 0.4385, 'grad_norm': 5.1204609870910645, 'learning_rate': 5e-05, 'epoch': 0.4}


 27%|██▋       | 1000/3750 [01:56<05:11,  8.82it/s]

{'loss': 0.3568, 'grad_norm': 3.3301167488098145, 'learning_rate': 4.230769230769231e-05, 'epoch': 0.8}


...

{'loss': 0.0779, 'grad_norm': 0.04976089671254158, 'learning_rate': 3.846153846153847e-06, 'epoch': 2.8}


                                                   
100%|██████████| 3750/3750 [07:46<00:00,  8.04it/s]


{'eval_loss': 0.4847284257411957, 'eval_accuracy': 0.8902, 'eval_runtime': 9.7618, 'eval_samples_per_second': 512.199, 'eval_steps_per_second': 8.093, 'epoch': 3.0}
{'train_runtime': 466.4496, 'train_samples_per_second': 128.631, 'train_steps_per_second': 8.039, 'train_loss': 0.2293516896565755, 'epoch': 3.0}


100%|██████████| 79/79 [00:09<00:00,  8.28it/s]

Accuracy: 0.8902
Classification Report:
              precision    recall  f1-score   support

         neg       0.89      0.89      0.89      2446
         pos       0.89      0.89      0.89      2554

    accuracy                           0.89      5000
   macro avg       0.89      0.89      0.89      5000
weighted avg       0.89      0.89      0.89      5000

